In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:00 #3 [Debug] buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:00 #4 [Debug] sendJson / port: 13805 / json: {"FileOpen":{"spiText":"inl app () =\r\n    \u0022test\u0022 |\u003E console.write_line\r\n    0i32\r\n\r\ninl main () =\r\n    print_static \u0022\u003Ctest\u003E\u0022\r\n\r\n    app\r\n    |\u003E dyn\r\n    |\u003E ignore\r\n\r\n    print_static \u0022\u003C/test\u003E\u0022\r\n","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result.Length:
00:00:00 #5 [Debug] sendJson / port: 13805 / json: {"BuildFile":{"backend":"Fsharp","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result.Length:
00:00:00 #6 [Debug] buildFile / takeWhileInclusive / fsxContent: let rec closure0 () () : int32 =
    let v0 : string = "test"
    System.Console.WriteLin

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.14.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @johannesegger
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Retrieving project options from cache, in case of issues run `dotnet fable clean` or try `--noCache` option.
Project and references (1 source files) parsed in 190ms

Started Fable compilation...

Fable compilation finished in 1410ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> apps\spiral\temp\test\./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

   --> apps\spiral\temp\test\./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^

  --> apps\spiral\temp\test\./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&m

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> apps\spiral\temp\test\./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

    Finished `release` profile [optimized] target(s) in 26.35s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\spiral_temp_test-b035f1bd4eb25d0e.exe)

running 3 tests
test test_parse_number ... ok
2024-03-18T10:56:28.828394Z  INFO spiral_temp_test: input=Integer(-3790664577244857790)
2024-03-18T10:56:28.828536Z  INFO spiral_temp_test: input=Comment("

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(GetExecutableSuffix) } | Invoke-Block

app=test
